# Generation of APEX Weather Files
This notebook contains scripts to format climate data to .dly and .hly files for APEX.

In [273]:
import os
import requests
import pandas as pd
from io import StringIO
import datetime

def convert_julian(year,doy):
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=doy - 1)
    return date.month, date.day

## From NASA POWER

### Load data using API

In [291]:
# set parameters for API search

# edit these
timestep = 'daily' # make sure that param_subset matches
longitude = '-80.0598'
latitude = '40.443'
start = '20130101'
end = '20221231'
elevation = '372.8'

# data needed for hly and dly files
params_hly = [
    'PRECTOTCORR'
]
    
params_dly = [
    'T2M_MAX',
    'T2M_MIN',
    'RH2M',
    'ALLSKY_SFC_SW_DWN',
    'PRECTOTCORR',
    'WS10M'
]
# USE PARAMS
param_subset = ','.join(params_dly) # change argument based on timestep used

In [293]:
url = f'https://power.larc.nasa.gov//api/temporal/{timestep}/point?parameters={param_subset}&community=AG&longitude={longitude}&latitude={latitude}&start={start}&end={end}&format=CSV'

response = requests.get(url)

In [294]:
data = response.text
print(data)

-BEGIN HEADER-
NASA/POWER Source Native Resolution Daily Data 
Dates (month/day/year): 01/01/2013 through 12/31/2022 in LST
Location: latitude  40.443   longitude -80.0598 
elevation from MERRA-2: Average for 0.5 x 0.625 degree lat/lon region = 332.12 meters
The value for missing source data that cannot be computed or is outside of the sources availability range: -999 
parameter(s): 
T2M_MAX               MERRA-2 Temperature at 2 Meters Maximum (C) 
T2M_MIN               MERRA-2 Temperature at 2 Meters Minimum (C) 
RH2M                  MERRA-2 Relative Humidity at 2 Meters (%) 
ALLSKY_SFC_SW_DWN     CERES SYN1deg All Sky Surface Shortwave Downward Irradiance (MJ/m^2/day) 
PRECTOTCORR           MERRA-2 Precipitation Corrected (mm/day) 
WS10M                 MERRA-2 Wind Speed at 10 Meters (m/s) 
-END HEADER-
YEAR,DOY,T2M_MAX,T2M_MIN,RH2M,ALLSKY_SFC_SW_DWN,PRECTOTCORR,WS10M
2013,1,-1.38,-13.31,99.47,1.72,1.9,1.96
2013,2,-5.88,-15.15,97.25,7.02,0.0,1.84
2013,3,-2.6,-17.03,97.97,7.24,0.0,

### .DLY format

In [277]:

def nasa2dly(nasa_data, dly_fp, dly_fn, wpxpm_fp, hly, spinup):
    
    '''
    formats nasa power data to APEX .dly
    
    parameters:
        nasa_data: data downloaded using API above
        output_fp: directory path to output .dly
        output_fn: filename for output .dly 
        wpxpm_fp: for monthly (wp1) generation, fp to wxpm executable ("None" to override)
                - note: file must contain both wxpm.exe and wxpmrun.DAT
        spinup[y/n]: y duplicates first year of data 4x to account for model spinup time
    ''' 
    # read data into df
    dly_df = pd.read_csv(StringIO(nasa_data), delimiter=',', skiprows=14)
    
    # add columns MONTH and DAY
    dly_df['MONTH'], dly_df['DAY'] = zip(*dly_df.apply(lambda row: convert_julian(int(row['YEAR']), int(row['DOY'])), axis=1))
    
    # set column names to match .dly
    dly_colnames = {'T2M_MAX':'TMAX',
                    'T2M_MIN':'TMIN',
                    'PRECTOTCORR':'PRCP',
                    'RH2M':'RH',
                    'ALLSKY_SFC_SW_DWN':'SRAD',
                    'WS10M':'WSPD'}
    dly_df = dly_df.rename(columns=dly_colnames)
    
    # rearrange columns
    dly_df = dly_df[['YEAR','MONTH','DAY','SRAD','TMAX','TMIN','PRCP','RH','WSPD']]
    
    # write .dly file
    with open(dly_fn, 'w') as file:
        for _, row in dly_df.iterrows():
            file.write(f'{int(row['YEAR']):6d} {int(row['MONTH']):3d} {int(row['DAY']):3d} {float(row['SRAD']):5.1f} {float(row['TMAX']):5.1f} {float(row['TMIN']):5.1f} {float(row['PRCP']):5.1f} {float(row['RH']):5.1f} {float(row['WSPD']):5.1f}\n')

    # generate wp1 file
    if wpxm_fp != None:
        dly2wp1(dly_fp, dly_fn, wxpm_fp)

    # generate .hly file via equal disaggregation





nasa_data = data
dly_fp = '/Users/mayastruzak/Desktop'
dly_fn = '/test2.dly'
wpxpm_fp = 
nasa2dly(nasa_data, output_dly)

In [300]:
dly_df = pd.read_csv(StringIO(nasa_data), delimiter=',', skiprows=14)

def dly2wp1(dly_fp, dly_fn, wxpm_fp):
    
    '''
    converts .dly file to .wp1 using wxpm
    
    parameters:
        dly_fp: path to file containing source .dly (and destination for wp1)
        dly_fn: .dly filename
        wxpm_fp: path to file containing wxpmrun.dat and wxpm.exe
    '''
    
    # set file paths
    src_dly = os.path.join(dly_fp, dly_fn)
    wxpm_dly = os.pathjoin(wxpm_fp, dly_fn)
    wxpm_run = os.path.join(wxpm_fp, 'wxpmrun.dat')
    wxpm_exe = os.path.join(wxpm_fp, 'wxpmrun.exe')
    wp1_fn = os.path.splitext(dly_fn)[0] + '.wp1'
    out_wp1 = os.path.join(dly_fp, wp1_fn)

    # copy DLY file into WXPM folder
    shutil.copy(src_file, wxpm_dly)

    # update wxpmrun.dat
    yr1 = pd.read_csv(dly_df)['year'].min() # get start year of data
    base_name = os.path.splitext(src_dly)[0] # get name of dly file without extension
    with open(wxpm_run, 'r') as f:
        lines = f.readlines()
    lines[0] = f"{src.dly} {yr1)\n"
    with open(wxpm_run, 'w') as f:
        f.writelines(lines)

    # run wxpm.exe
    subprocess.run([wxpm_exe], cwd=wxpm_fp)

    # copy generated wp1 to dly folder
    shutil.copy(os.path.join(wxpm_fp, wp1_fn), out_wp1)


SyntaxError: unterminated string literal (detected at line 26) (3110833107.py, line 26)

#### .DLY with spinup time (4x first year)

In [ ]:
df = 

### .HLY format

In [ ]:
hly_df = pd.read_csv(